In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
18,application_1612044880670_0019,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hsfs

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store("amlsim_featurestore")

Connected. Call `.close()` to terminate connection gracefully.

In [4]:
eval_df = fs.get_training_dataset("gan_eval_df", 1).read()
train_df = fs.get_training_dataset("gan_non_sar_training_df", 1).read() 
emb_df = eval_df.union(train_df) #.where(eval_df.target==1)
emb_df.show()

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     1|[0.48607898, -0.1...|
|     1|[-0.47727537, 0.1...|
|     1|[0.22666311, -0.5...|
|     1|[-0.7683222, 0.13...|
|     1|[0.39057684, 0.96...|
|     1|[-0.7735877, 0.05...|
|     1|[-0.095639706, 0....|
|     1|[0.47294903, -0.3...|
|     1|[-0.39901948, -0....|
|     1|[0.7641628, 0.305...|
|     1|[0.69657683, 0.11...|
|     1|[-0.0423429, -0.6...|
|     1|[0.868845, 0.0528...|
|     1|[-0.43252254, 0.5...|
|     1|[0.06834984, -0.7...|
|     1|[-0.47181463, -0....|
|     1|[-0.20204735, -0....|
|     1|[0.80187464, 0.07...|
|     1|[0.4038105, 0.544...|
|     1|[0.9727471, 0.077...|
+------+--------------------+
only showing top 20 rows

In [5]:
emb_df.where(emb_df.target == 0).show()

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     0|[-0.7185197, -0.5...|
|     0|[-0.5465119, 0.73...|
|     0|[-0.2560954, 0.67...|
|     0|[0.03640437, 0.76...|
|     0|[-0.9657593, -0.7...|
|     0|[-0.6548383, -0.3...|
|     0|[0.18099833, 0.23...|
|     0|[-7.095337E-4, -0...|
|     0|[0.087100744, -0....|
|     0|[0.5484538, -0.84...|
|     0|[-0.678113, -0.36...|
|     0|[-0.29078293, -0....|
|     0|[0.12922144, 0.77...|
|     0|[-0.5990057, -0.0...|
|     0|[-0.31415105, 0.5...|
|     0|[-0.2342267, -0.8...|
|     0|[-0.43512964, -0....|
|     0|[-0.2187376, -0.3...|
|     0|[0.10286474, -0.4...|
|     0|[-0.89329696, 0.2...|
+------+--------------------+
only showing top 20 rows

In [6]:
# Get fg as pandas
from pyspark.sql.functions import explode
dlist = emb_df.columns
emb_pdf = emb_df.select(dlist+[(emb_df.embedding[x]).alias("Value"+str(x+1)) for x in range(0, 128)]).drop("embedding").toPandas()

In [7]:
#%%spark -o emb_pdf

In [8]:
emb_pdf.head

<bound method NDFrame.head of       target    Value1    Value2  ...  Value126  Value127  Value128
0          1  0.486079 -0.189078  ... -0.837726  0.279952 -0.123464
1          1 -0.477275  0.147429  ...  0.718477  0.100887  0.402870
2          1  0.226663 -0.533207  ... -0.564404 -0.305063 -0.916829
3          1 -0.768322  0.132927  ...  0.296606  0.620593 -0.176689
4          1  0.390577  0.961654  ... -0.193944  0.149571  0.569036
...      ...       ...       ...  ...       ...       ...       ...
7342       0  0.699275 -0.162821  ...  0.535009 -0.311959  0.359504
7343       0  0.726155  0.538397  ... -0.363945 -0.011806 -0.075307
7344       0  0.816660  0.897723  ... -0.815421 -0.554857 -0.909673
7345       0  0.936104  0.153948  ... -0.073009  0.574821 -0.092643
7346       0  0.963763 -0.816191  ...  0.546268 -0.630459 -0.368822

[7347 rows x 129 columns]>

In [9]:
#%%local 

import os
import sys
import uuid
import random    
    
import numpy as np
import pandas as pd

import pydoop.hdfs as pydoop

import matplotlib.pyplot as plt
    
from sklearn.manifold import TSNE

from hops import hdfs


walk_number = 40 
walk_length = 5 
emb_size = 128

node_embeddings = emb_pdf.loc[:, emb_pdf.columns != 'target']
target = emb_pdf.target
embeddings_features = pd.DataFrame(node_embeddings, index=emb_pdf.target.values)
print(len(embeddings_features.index.values))

def _tsne_plot(node_embeddings, walk_number, walk_length, emb_size):
        transform = TSNE  # PCA
        trans = transform(n_components=2, random_state=123)        
        #trans = transform(n_components=2, verbose=1, perplexity=40, n_iter=1500)
        
        node_embeddings_2d = trans.fit_transform(node_embeddings)


        # draw the embedding points, coloring them by the target label (paper subject)
        alpha = 0.7

        """
        ###############
        labels = pd.read_csv(pydoop.path.abspath(hdfs.get_plain_path([path for path in hdfs.ls(labels_file) if path.endswith("csv")][0])))
        label_map = pd.Series(labels.is_sar.values,index=labels.id).to_dict()
        node_colours = []
        for target in node_embeddings.index.values:
            if target in label_map:
                node_colours.append(label_map[target])
            else:
                node_colours.append(0)
        ################        
        """
        node_colours = node_embeddings.index.values

        plt.figure(figsize=(300, 300))
        plt.axes().set(aspect="equal")
        plt.scatter(
            node_embeddings_2d[:, 0],
            node_embeddings_2d[:, 1],
            c=node_colours,
            cmap="jet",
            alpha=alpha,
        )
        #plt.title("{} visualization of node embeddings".format(transform.__name__))
        tsne_file = 'node_embeddings_features_%d_%d_%d.tsne.pdf' % (walk_number, walk_length, emb_size)
        plt.savefig(tsne_file)
        plt.close()
        hdfs.copy_to_hdfs(tsne_file, "Resources", overwrite=True, project="devaml")
        #plt.show()
    
_tsne_plot(embeddings_features, walk_number, walk_length, emb_size)

An error was encountered:
Unexpected character ('#' (code 35)): expected a valid value (number, String, array, object, 'true', 'false' or 'null')
 at [Source: #; line: 1, column: 2]

